In [1]:
import os
import sys
import pandas as pd
import librosa
import librosa.display
import numpy as np
import matplotlib.pyplot as plt
from pydub import AudioSegment
import traceback
from utils import map_images_to_classes

In [ ]:
# make csv file for AST Feature Extractor (audio segments(10xsong)and valence classes)

# create function that splits data train-test-validate based on df(csv) and extracts equal parts of each class---) new dfs

# create function that does the same thing but like a dataloader --) nothing new created, only associations

In [2]:
dfg = pd.read_csv('../data/processed/1000dataset_valence_cluster.csv')


In [3]:
#count unique values in column 'valence_cluster'
valence_cluster = dfg['valence_cluster'].value_counts()
print(valence_cluster)

valence_cluster
high      386
medium    314
low       300
Name: count, dtype: int64


In [5]:
dfg = pd.read_csv('../data/processed/1000dataset_valence_cluster.csv')
spec_df_danceability = map_images_to_classes('../data/processed/1000dataset_5/specs', dfg, 'track_id', 'valence_cluster')

In [6]:
# label encode the class
from sklearn.preprocessing import LabelEncoder
labelencoder = LabelEncoder()
spec_df_danceability['class'] = labelencoder.fit_transform(spec_df_danceability['class'])

In [8]:
#spec_df_danceability.head(6)
# save it to csv
spec_df_danceability.to_csv('../data/processed/L1000dataset_5seg_valence.csv', index=False)

Keep only audio in 'audio' folder

In [6]:
# run move_json_files function
#move_json_files()

Transoform audio to spectograms

In [57]:
import random
import os
import librosa
import librosa.display
import numpy as np
import matplotlib.pyplot as plt
from pydub import AudioSegment
import gc
import traceback


def mp3_to_melspectrogram(mp3_file, audio_output_folder, spec_output_folder, trim_seconds=30,
                          include_legend=False, nmels=512, segment_duration=5, save_audio_segments=True,
                          num_segments=5):
    """
    Converts an MP3 file to mel spectrograms and optionally saves the audio segments.

    Parameters:
        mp3_file (str): Path to the MP3 file.
        audio_output_folder (str): Folder to save the audio segments.
        spec_output_folder (str): Folder to save the spectrogram images.
        trim_seconds (int): Number of seconds to trim from both ends of the audio.
        include_legend (bool): Whether to include legends and axes in the spectrogram plots.
        nmels (int): Number of mel bands to generate in the spectrogram.
        segment_duration (int): Duration of each segment in seconds. If None, processes the entire audio.
        save_audio_segments (bool): Whether to save the corresponding audio segments.
        num_segments (int or None): Number of segments to save. If None, saves all segments.
    """
    # Ensure both output folders exist
    os.makedirs(audio_output_folder, exist_ok=True)
    os.makedirs(spec_output_folder, exist_ok=True)

    try:
        print(f"Loading file: {mp3_file}")

        # Load the MP3 file using PyDub
        audio = AudioSegment.from_file(mp3_file)
        total_duration_ms = len(audio)  # Duration in milliseconds
        total_duration = total_duration_ms / 1000.0  # Convert to seconds
        print(f"File {mp3_file}, Total Duration: {total_duration:.2f} seconds")

        # Check if the file is long enough to trim
        if total_duration <= 2 * trim_seconds:
            print(f"File {mp3_file} is too short to trim {trim_seconds} seconds from both ends. Skipping.")
            return

        # Trim the first and last trim_seconds
        start_trim = trim_seconds * 1000  # Convert to milliseconds
        end_trim = total_duration_ms - trim_seconds * 1000
        audio_trimmed = audio[start_trim:end_trim]

        print(f"Trimmed audio length (ms): {len(audio_trimmed)}")

        # Calculate segment length in samples if segment_duration is specified
        sr = audio_trimmed.frame_rate
        samples_per_segment = int(segment_duration * sr) if segment_duration else len(audio_trimmed.get_array_of_samples())

        # Convert to NumPy array
        y = np.array(audio_trimmed.get_array_of_samples()).astype(np.float32)

        # Convert stereo to mono
        if audio_trimmed.channels == 2:
            y = y.reshape((-1, 2)).mean(axis=1)

        print(f"Sample rate: {sr}, Audio shape after conversion: {y.shape}")

        # Determine the total number of segments
        total_segments = len(y) // samples_per_segment

        # Select random segments if num_segments is specified
        if num_segments is not None and num_segments < total_segments:
            selected_indices = sorted(random.sample(range(total_segments), num_segments))
        else:
            selected_indices = list(range(total_segments))

        print(f"Total segments: {total_segments}, Selected segments: {len(selected_indices)}")

        # Process each selected segment
        for segment_count, idx in enumerate(selected_indices, start=1):
            start_sample = idx * samples_per_segment
            segment = y[start_sample:start_sample + samples_per_segment]

            # Generate the mel spectrogram
            mel_spectrogram = librosa.feature.melspectrogram(y=segment, sr=sr, n_mels=nmels)
            D = librosa.power_to_db(mel_spectrogram, ref=np.max)

            # Plot and save the mel spectrogram
            plt.figure(figsize=(10, 6))
            librosa.display.specshow(D, sr=sr, x_axis='time' if include_legend else None,
                                     y_axis='mel' if include_legend else None, cmap='inferno')

            if include_legend:
                plt.colorbar(format='%+2.0f dB')  # Add color bar if include_legend is True
                plt.title(f'Mel Spectrogram of {os.path.basename(mp3_file)} - Segment {segment_count}')  # Add title
            else:
                plt.axis('off')  # Turn off axes if include_legend is False
                plt.gca().set_position([0, 0, 1, 1])  # Remove any padding/margins

            # Save the spectrogram image
            output_image_path = os.path.join(
                spec_output_folder, f"{os.path.splitext(os.path.basename(mp3_file))[0]}_segment_{segment_count}.png"
            )
            plt.savefig(output_image_path, bbox_inches='tight', pad_inches=0 if not include_legend else 0.1)
            plt.close()

            print(f"Spectrogram saved to {output_image_path}")

            # Save the corresponding audio segment if required
            if save_audio_segments:
                audio_segment = audio_trimmed[start_sample // sr * 1000:(start_sample + samples_per_segment) // sr * 1000]
                audio_segment_path = os.path.join(
                    audio_output_folder, f"{os.path.splitext(os.path.basename(mp3_file))[0]}_segment_{segment_count}.mp3"
                )
                audio_segment.export(audio_segment_path, format="mp3")
                print(f"Audio segment saved to {audio_segment_path}")

        # Explicitly release memory
        del audio, audio_trimmed, y, segment
        gc.collect()

    except Exception as e:
        print(f"Error processing {mp3_file}: {e}")
        traceback.print_exc()  # Print detailed error traceback
        
def process_all_mp3_in_folder(folder_path, audio_output_folder, spec_output_folder):
    # Ensure the output folders exist
    os.makedirs(audio_output_folder, exist_ok=True)
    os.makedirs(spec_output_folder, exist_ok=True)

    # Loop through all MP3 files in the folder
    for file in os.listdir(folder_path):
        if file.endswith(".mp3"):
            mp3_file = os.path.join(folder_path, file)

            # Check if the first spectrogram segment exists
            first_segment_path = os.path.join(
                spec_output_folder, f"{os.path.splitext(file)[0]}_segment_1.png"
            )
            if os.path.exists(first_segment_path):
                print(f"Skipping {mp3_file}, spectrogram already exists.")
                continue

            print(f"Processing file: {mp3_file}")
            mp3_to_melspectrogram(mp3_file, audio_output_folder, spec_output_folder)


In [ ]:
process_all_mp3_in_folder("../data/raw/1000dataset", "../data/processed/1000dataset_5/audio", "../data/processed/1000dataset_5/audio_specs")

In [ ]:
# Example usage for processing multiple MP3 files
folder_path = "../data/raw/1000dataset"  # Folder containing MP3 files
#output_folder = "../data/raw/1000_mel_spectrograms"  # Folder to save spectrogram images
output_folder = "../data/processed/1000_melspec_5seg_png"  # Folder to save segmented spectrogram images
process_all_mp3_in_folder(folder_path, output_folder)


legend and segment parameters

# Transform text to jsonl form


In [ ]:
# {"file_name" : "3odrUVQ9tvRpkC9II2oWzx_segment_5.png", "additional_feature": "Ellie Goulding electro"}
# So if the str before the first underscore is the same to the row track_id column, then add text to the additional feature column

{'file_name': '3odrUVQ9tvRpkC9II2oWzx_segment_5.png',
 'additional_feature': 'Ellie Goulding electro'}

SEGMENT SPECTOGRAM


Transform audio and segment spectograms


Transform spectogram to audio

In [ ]:
import numpy as np
import librosa
import librosa.display
from pydub import AudioSegment

# Function to convert spectrogram to audio and save it as MP3
def spectrogram_to_mp3(spec, sr, filename="output.mp3"):
    # Step 1: Invert the spectrogram to a time-domain audio signal
    # If the spectrogram is in dB, first convert back to amplitude
    if np.max(spec) < 0:  # Checking if it's in dB scale
        spec = librosa.db_to_amplitude(spec)
    
    # If the spectrogram is a magnitude spectrogram, use librosa's inverse STFT
    audio_signal = librosa.istft(spec)
    
    # Step 2: Save the audio signal as a WAV file
    librosa.output.write_wav("temp.wav", audio_signal, sr)
    
    # Step 3: Convert WAV to MP3 using pydub
    sound = AudioSegment.from_wav("temp.wav")
    sound.export(filename, format="mp3")
    
    print(f"Saved MP3 as {filename}")

# Example usage
if __name__ == "__main__":
    # Load an example audio file to create a spectrogram
    y, sr = librosa.load(librosa.example('trumpet'))
    
    # Generate a spectrogram from the audio signal
    spec = librosa.stft(y)
    spec_db = librosa.amplitude_to_db(np.abs(spec), ref=np.max)
    
    # Convert the spectrogram back to MP3
    spectrogram_to_mp3(spec_db, sr, filename="output.mp3")


CHECK FROM SPECTOGRAM TO MP3

In [18]:
image_path = "raw/spectrograms/0A0RBBTrgfq9eClnw6ZXT7.png"


In [ ]:
import numpy as np
import librosa
from PIL import Image
from pydub import AudioSegment
import soundfile as sf  # For saving audio files

def image_to_spectrogram(image_path):
    """
    Convert a spectrogram image to a numerical array representing the spectrogram.
    Assumes the spectrogram is grayscale, where pixel intensity represents dB values.
    """
    # Load the image
    img = Image.open(image_path).convert("L")  # Convert to grayscale ('L' mode)
    
    # Convert image to numpy array
    img_array = np.array(img)
    
    # Normalize the image to [0, 1] range (0 is minimum, 255 is maximum)
    img_array = img_array / 255.0

    # Convert the image intensities to dB scale (assuming they represent dB)
    # Rescale from [0, 1] to [-80 dB, 0 dB], as typical dB range for spectrograms is -80 to 0
    img_db = img_array * -80
    
    return img_db

def spectrogram_to_audio(spectrogram_db, sr=22050, hop_length=512):
    """
    Invert a dB-scaled spectrogram back to an audio waveform.
    """
    # Convert dB to amplitude
    spectrogram_amplitude = librosa.db_to_amplitude(spectrogram_db)
    
    # Perform the inverse Short-Time Fourier Transform (ISTFT) to recover the audio signal
    audio_signal = librosa.griffinlim(spectrogram_amplitude, hop_length=hop_length)
    
    return audio_signal

def save_as_mp3(audio_signal, sr, filename="output.mp3"):
    """
    Save the audio signal as an MP3 file using pydub.
    """
    # Save the audio signal as a temporary WAV file using soundfile
    sf.write("temp.wav", audio_signal, sr)  # Use soundfile to save as WAV
    
    # Convert the WAV file to MP3 using pydub
    sound = AudioSegment.from_wav("temp.wav")
    sound.export(filename, format="mp3")
    print(f"Saved MP3 as {filename}")

# Example usage
if __name__ == "__main__":
    # Path to the spectrogram image (PNG)
    image_path = "raw/spectrograms/0A0RBBTrgfq9eClnw6ZXT7.png"
    
    # Step 1: Convert the image to a spectrogram
    spectrogram_db = image_to_spectrogram(image_path)
    
    # Step 2: Convert the spectrogram to an audio signal
    audio_signal = spectrogram_to_audio(spectrogram_db, sr=22050)
    
    # Step 3: Save the audio signal as MP3
    save_as_mp3(audio_signal, sr=22050, filename="output.mp3")


In [1]:
import torch
torch.cuda.empty_cache()


audio shape ???
sample rate ???

Download everything in wav? or find a solution in another way ?